## Distributed Data Parallel Frameworks

We now turn to distributed data parallel frameworks.  The next section will take us through the evolution of Map/Reduce -> Hadoop! -> Spark.

## Limitations of Dask: Why to use Spark!

Dask is well integrated into the python ecosystem. This is the best and worst part of dask.  
* Best:
  * dask inherites the efficient implementations of the underlying libraries in NumPy and pandas.
  * dask is easy to use for python programers
* Worst
  * dask is python and python is a serial and interpreted (not compiled) language
  * inefficient for user-defined functions (they run in python)
  * global shuffles are inefficient in dask
  
Shuffles are the key.  Map/reduce is about shuffling (sorting data by key). All systems that build on top of this paradigm inherit efficient shuffles in distributed memory.

### Dask Datatypes, Functions, and Operators

Because Dask is a data parallel language, it's reasonable to categorize dask around the three major "collections" implemented:
  * dask.array: a parallel NumPy array
  * dask.dataframe: a parallel pandas dataframe
  * dask.bag: inherited from Spark (and Pig).
  
So, arrays and dataframes make sense.  Where did this bag come from? Dask reports that "It is similar to a parallel version of PyToolz or a Pythonic version of the PySpark RDD."

### dask bags

A dask bag or multiset is:
  * unordered: cannot be indexed like an array
  * not-unique: can have repeated entries
  * contains arbitrary python objects
  
This makes it a **key/value** system
  * key: identifier for a data object that can be evaluated for equality (and typically sorted)
  * value: an uninterpreted _BLOB_ of data. The system can't unpack or operate on value data.  Although, user-defined functions can.
  
The dask guidance is to only use bags when absolutely needed and to convert to arrays or dataframes as soon as possible. Bags support the nested data structures typical of JSON, e.g. dictionaries that contain lists of lists.  The limitations are:
  * bags only use the 'processes' scheduler and cannot share memory among the multiple cores of a node
  * user-defined functions are inefficient when compared with pandas or numpy builtins 

Additionally, dask **strongly** encourages you to avoid <code>bag.groupby()</code>, because it requires a full shuffle (sort by key) of the data.

**Conclusion**: dask bags are for computing user-defined functions on distributed data structures. But, these are not dask's strengths.  In this case, we will look to other engines.

## A brief history of data-parallel compute engines

* Map/Reduce (2004): At Google, Jeff Dean and Sanjay Ghemawat outline the future of large-scale data processing
  * <code>map()</code>: applies a user-defined function to a data partition and outputs a <code>key</code> used to identify objects in a class and a <code>value</code> that contains the data.
  * <code>reduce()</code>: takes all items with the same key and applies a user-defined function that aggregates the data.
  * This was the start of automatic parallelism (some disagree). The programmer writes two _pure_ functions and creates a computation that scales to thousands of nodes and TB of data.
* Hadoop! (2006): open source implementation of map/reduce computing
  * Credit to Doug Cutting and Mike Cafarella.
  * Users write Java functions that execute at scale.
  
We now move out of the Google ecosystem. Google has continued to make important contributions that inform open-source.

* Pig (2008): meta-programming for Hadoop!
  * declarative constructs that compile to map/reduce programs
  * uses the bag data type as an abstraction for key/value data 
* Hive (2010): SQL interface to Hadoop!
  * SQL queries can be executed in two iterations of map/reduce
  * Scalable data processing for those
* Spark (2014): in-memory data for iterative programming in map/reduce
  * built on the "resilient distributed dataset" which is a data-parallel partitioned multiset.
  * executes programs on Hadoop!
* Dask (2016?): "pythonic" version of Spark that eases programming for NumPy and pandas.

### Conclusions

Guidance for dask:
  * try to convert semi-structured data to dataframes as soon as possible
  * try to use built-in functions whenever possible, they are optimized
  
Guidance for when and how to use Spark:
  * for workloads that perform shuffles, Spark runs on top of the Hadoop! engine.
  * for complex user-defined functions, but you must write them in Scala which compiles into java.

Spark is a bigger, heavier ecosystem with a more complex distributed query optimizer.